In [1]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from ASC_ML.networkbuilding import multiple_model_gen_v3 as multiple
from ASC_ML.networkbuilding import dataframe_extractor as de
from ASC_ML.networkbuilding import model_generation as model_gen
from ASC_ML.networkbuilding import model_optimization as model_opt
# from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-08-19 20:53:17.983419: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# m = multiple.Multiple_Model_Gen(max_no_layers = 2, model_per_batch = 10)

In [4]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [5]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [6]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [7]:
train_x = dataset_list[0].copy()

In [8]:
train_x = train_x.fillna(0)

In [9]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [10]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
# a = train_x['LotShape']

In [12]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [13]:
# train_x.head()

In [14]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [15]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [16]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [17]:
train_X = train_x.replace(keys).copy()

In [18]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [19]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [20]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [21]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [22]:
train_X = train_X.drop(["Id"], axis = 1)

In [23]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [24]:
len(train_X.columns)

58

In [25]:
len(dataset_list[1].columns)

1

In [26]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [27]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [28]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [29]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [30]:
print(train_X_scaled.shape)
print(train_Y_scaled.shape)

(1460, 58)
(1460, 1)


In [31]:
import numpy as np
total_data = np.append(train_X_scaled, train_Y_scaled, axis = 1)

In [32]:
print(total_data.shape)

(1460, 59)


In [33]:
from sklearn.model_selection import train_test_split
(train, test) = train_test_split(total_data, test_size=0.25, random_state=42)

In [34]:
trainX = train[:,:58]
trainY = train[:,-1]
testX = test[:,:58]
testY = test[:,-1]
print(trainX.shape)
print(trainY.shape)

(1095, 58)
(1095,)


In [35]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [36]:
trainY = trainY.reshape((trainY.shape[0],1))
print(trainY.shape)

(1095, 1)


In [37]:
testY = testY.reshape((testY.shape[0],1))

In [38]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(trainX, trainY, testX, testY, 25, 64, input_shape = 58, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models(save = True)
end = time.time()
print("Time Taken : ", (end-start))

2022-08-19 20:53:31.367941: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-19 20:53:31.369696: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-19 20:53:31.438855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-19 20:53:31.439114: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-08-19 20:53:31.439146: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-19 20:53:31.441657: I tensorflow/stream_executor/platform/de


 
 

loss  :  0.007878750562667847 , TEST :  0.02319185622036457
output_layer_dense_32_32_loss  :  0.0016125185647979379 , TEST :  0.0026418566703796387
output_layer_dense_32_64_loss  :  0.001104635070078075 , TEST :  0.002844856819137931
output_layer_dense_32_128_loss  :  0.001237591844983399 , TEST :  0.0027227287646383047
output_layer_dense_64_32_loss  :  0.0008777790935710073 , TEST :  0.002574292942881584
output_layer_dense_64_64_loss  :  0.0006379507831297815 , TEST :  0.002432562643662095
output_layer_dense_64_128_loss  :  0.0005192172829993069 , TEST :  0.0024992763064801693
output_layer_dense_128_32_loss  :  0.0009468565112911165 , TEST :  0.0024560161400586367
output_layer_dense_128_64_loss  :  0.0006178499897941947 , TEST :  0.002591267693787813
output_layer_dense_128_128_loss  :  0.0003243505780119449 , TEST :  0.002429000101983547

 
 

loss  :  0.008214936591684818 , TEST :  0.023154329508543015
output_layer_dense_32_32_32_loss  :  0.0013287761248648167 , TEST :  0.00263

2022-08-19 20:53:46.908934: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_128_128_32/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_64_64_64/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_64_64_128/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_64_32/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_64_128/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_128_64/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_32_128/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_64_128_128/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_128_64_32/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_128_32/assets

-------------------------------------------------------------------------------
Time Taken :  18.66

In [39]:
opt = model_opt.Model_Optimization(trainX, trainY, testX, testY, 80, m.evaluate_dict_list)
# opt.optimize_models()
# opt.train_models_2()

In [ ]:
opt.optimize_models()

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 0.0028706183657050133, BEST_ACTIVATION : selu, BEST_INITIALIZER : GlorotUniform, BEST_LEARINING_RATE : 0.01585, BEST_BATCHSIZE : 16

 
 

loss  :  3.8610622882843018 , TEST :  3.845639228820801
----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 0.0016618164954707026, BEST_ACTIVATION : relu, BEST_INITIALIZER : GlorotNormal, BEST_LEARINING_RATE : 0.002, BEST_BATCHSIZE : 16

 
 

loss  :  0.00011504268331918865 , TEST :  0.0019520710920915008
----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 0.0040299068205058575, BEST_ACTIVATION : selu, BEST_INITIALIZER : GlorotUniform, BEST_LEARINING_RATE : 0.00126, BEST_BATCHSIZE : 16

 
 

loss  :  0.00060235516866

In [ ]:
a = 1
b = [a]
print(b)

In [ ]:
a = (int)t

In [ ]:
import tensorflow
model = tensorflow.keras.models.load_model("/home/anish/ASC_ML_test_weights/dense_32_32_64")

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr = 1e-3)
model.compile(loss = "mean_squared_error", optimizer = optimizer)
history = model.fit(trainX, trainY, epochs = 80, batch_size = 64)

In [ ]:
for model_dict in m.evaluate_dict_list:
    print("--------------------------------------------------------------------------------")
    model = tensorflow.keras.models.load_model(model_dict["path_weights"])
    optimizer = Adam(lr = 1e-3)
    model.compile(loss = "mean_squared_error", optimizer = optimizer)
    history = model.fit(trainX, trainY, epochs = 80, batch_size = 16)

In [ ]:
a = (int)t

In [ ]:
from tensorflow.keras.activations import tanh, relu

## 2,3 Hidden Layers, 100 epochs, time taken = 147s (model saved)

In [ ]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(trainX, trainY, testX, testY, 10, 128, input_shape = 58, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models()
end = time.time()
print("Time Taken : ", (end-start))

In [ ]:
m.evaluate_dict_list